In [ ]:
# imports
import arcpy
import pandas as pd
import numpy as np

In [ ]:
# store data locations
counties = "C:\\Users\\2020c\\Documents\\GIS\\PublicSchoolProject\\PublicSchoolProject\\PublicSchoolProject.gdb\\counties"
edu_outcomes = "C:\\Users\\2020c\\Documents\\GIS\\PublicSchoolProject\\PublicSchoolProject\\PublicSchoolProject.gdb\\Edu_Outcomes"
budget = "C:\\Users\\2020c\\Documents\\GIS\\PublicSchoolProject\\PublicSchoolProject\\elsec20.csv"
grouped_outcomes = "C:\\Users\\2020c\\Documents\\GIS\\PSProject\\PSProject\\grouped_outcomes.csv"

In [ ]:
# clean edu_outcomes

# add geo_type field identifier
arcpy.management.AddField(edu_outcomes, "geo_type", "TEXT")
geo_type_cb = '"state" if !FIPS!.endswith("000") else "county"'
arcpy.management.CalculateField(edu_outcomes, "geo_type", geo_type_cb, "PYTHON3")

# replace null vals with -1
nulls = {'F2013_Urban_Influence_Code', 'F2013_Rural_urban_Continuum_Cod', 'F2003_Urban_Influence_Code', 'F2003_Rural_urban_Continuum_Cod'}
for f in nulls:
    null_cb = '-1 if not !' + f + '! else !' + f + '!'
    arcpy.management.CalculateField(edu_outcomes, f, null_cb, "PYTHON3")

In [ ]:
# join educational outcome data to county base layer
in_field = 'GEOID'
join_field='FIPS'
arcpy.management.AddJoin(counties, in_field, edu_outcomes, join_field)

In [ ]:
# read in and clean data
df = pd.read_csv(budget)
df.info()

x = df.isna().sum().sum()
print(x) # x = 0 -> no missing values

df

In [ ]:
# eliminate unnecessary columns that conflict with grouping
ind = [0, 1, 2, 4, 5, 6, 7, 8]
for i in ind:
    print(df.columns[i])

# rem = {"STATE", "IDCENSUS", "NAME", "CSA", "CBSA", "SCHLEV", "NCESID", "YRDATA"}
df = df.drop(df.columns[ind],axis = 1)

df

In [ ]:
# aggregation
cols = list(df.columns.values)
print(cols)
aggre = {}

for col in cols[1:]:
    aggre[col] = 'sum'

# print(aggre)

grouped = df.groupby("'CONUM'").agg(aggre)
print(grouped)

grouped

In [ ]:
grouped.to_csv(grouped_outcomes)
arcpy.management.AddJoin(counties, 'GEOID_WRA', grouped_outcomes, "'CONUM'")

In [ ]:
in_features = counties_upd_layer = "C:\\Users\\2020c\\Documents\\GIS\\PSProject\\PSProject\\PSProject.gdb\\counties_upd_export"
dependent_field = "Percent_of_adults_with_a_bachel"
sel_fields = [2]
explanatory_field = [cols[i] for i in sel_fields]
out_featureclass = "C:\\Users\\2020c\\Documents\\GIS\\PSProject\\PSProject\\geo_weights"
kernel_type = "ADAPTIVE"
bandwidth_method = "AICc"


arcpy.stats.GeographicallyWeightedRegression(in_features, dependent_field, explanatory_field, out_featureclass, kernel_type, bandwidth_method)